This baseline model 2 is based on the existing model and the data.csv (old data). Only the firms with less than 10 years of existence & with at least one fundraising.

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import json

import re
def load_json_field(bad_json):
    regex = r"\w\'\w"
    subst = ""
    bad_json = re.sub(regex, subst, bad_json)
    bad_json = bad_json.replace("d' Arrouzat", "darrouzat")
    good_json = bad_json.replace("\'", "\"").replace("None", "null").replace("True", "true").replace("False", "false")
    
    return json.loads(good_json)

data = pd.read_csv("bpideep/rawdata/data.csv")

# data preprocessing

## str -> json

In [3]:
data["team"] = data["team"].apply(load_json_field)
data["industries"] = data["industries"].apply(load_json_field)
data["investors"] = data["investors"].apply(load_json_field)
data["fundings"] = data["fundings"].apply(load_json_field)

## fill the missing values for the companies which have at least 1 fundraising

In [4]:
data.launch_year.isna().sum()

120

In [5]:
data[data.launch_year.isna()]["total_funding"].value_counts()

0.00     109
0.05       1
0.70       1
0.30       1
2.40       1
0.24       1
0.16       1
1.64       1
2.00       1
6.00       1
4.50       1
14.50      1
Name: total_funding, dtype: int64

In [6]:
names = list(data[data.launch_year.isna() & data.total_funding > 0 ]["name"])

In [7]:
# inserting missing launch year and month for the 11 companies with fundraising

launch_year = [2018, 2015, 2012, 2014, 2018, 2016, 2015, 2010, 2011, 2016, 2015]
launch_month = [5, 2, 11, 5, 9, 1, 1, 12, 9, 2, 9]

zipbObj = zip(names, launch_year)
launch_year_dict = dict(zipbObj)

zipbObj = zip(names, launch_month)
launch_month_dict = dict(zipbObj)

for name, year in launch_year_dict.items() : 
    data.loc[data.name == name, "launch_year"] = year
    
for name, month in launch_month_dict.items() : 
    data.loc[data.name == name, "launch_month"] = month   

/!\\
970400 : Institut de Prise en Charge de l'Obésité -> centre de prise en charge chirurgicale et médicale de l’obésité -> hors scope ?

## applying the filters : launch_year >= 100 & total_funding > 0

ATTENTION : the filter removes half of the dataset... 770 companies among which 270 deeptech

In [8]:
data.loc[(data.launch_year >= 2010) & (data.total_funding > 0)].shape

(748, 94)

In [9]:
data.loc[(data.launch_year < 2010) | (data.total_funding == 0)]["deep_or_not"].value_counts()

non_deeptech       379
deeptech           270
almost_deeptech    121
Name: deep_or_not, dtype: int64

In [10]:
data.loc[(data.launch_year < 2010) | (data.total_funding == 0)]["deep_or_not"].shape

(770,)

In [11]:
filtered_data = data.loc[(data.launch_year >= 2010) & (data.total_funding > 0)]

In [12]:
filtered_data["deep_or_not"].value_counts()

deeptech           377
non_deeptech       313
almost_deeptech     58
Name: deep_or_not, dtype: int64

Le model existant tel quel donne un X gardant la colonne "deep_or_not" -> enlève

In [13]:
X = filtered_data.drop(columns = ['target'])
y = filtered_data['target']

# Train_test_split

In [14]:
from sklearn.model_selection import train_test_split

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y)

#validation split
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify = y_train)

# Existing model

In [15]:
from bpideep.trainer import Trainer

model = Trainer(X_train_train, y_train_train)
model.train()
model.pipeline.score(X_val, y_val)

set_pipeline 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degr

/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['degree'] = data['team'].map(lambda x:degree(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['doctor_yesno'] = data['degree'].map(lambda x: degree_quant(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy 

0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_qua

/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['industry'] = data['industries'].map(lambda x: industries(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'] = pd.DataFrame({'year': data['launch_year']})
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:271: SettingWithCopyWarning: 
A value is trying to be set on a co

0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_rat

/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['degree'] = data['team'].map(lambda x:degree(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['doctor_yesno'] = data['degree'].map(lambda x: degree_quant(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy 

encoder 0.38
return_list 0.0
encoder 0.65
return_list 0.0
encoder 0.64
return_list 0.0
encoder 0.71
return_list 0.0
encoder 0.96
return_list 0.0
encoder 3.27
return_list 0.0
encoder 0.38
feat_eng_cols 7.15


0.7047619047619048

1st run : 0.7333333333333333

2nd run : 0.7047619047619048

In [16]:
from sklearn.model_selection import cross_validate

model2 = Trainer(X_train, y_train)
model2.train()

# Cross validate pipeline
cv_result = cross_validate(model2.pipeline, X_train, y_train, cv=5, scoring=["accuracy", "recall", "f1", "precision"])

set_pipeline 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degree 0.0
degr

degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0

/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['degree'] = data['team'].map(lambda x:degree(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['doctor_yesno'] = data['degree'].map(lambda x: degree_quant(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy 

growth_stage_num 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
industries 0.0
indu

/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['industry'] = data['industries'].map(lambda x: industries(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'] = pd.DataFrame({'year': data['launch_year']})
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:271: SettingWithCopyWarning: 
A value is trying to be set on a co

...

degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0.0
degree_quant 0

/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['industry'] = data['industries'].map(lambda x: industries(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'] = pd.DataFrame({'year': data['launch_year']})
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:271: SettingWithCopyWarning: 
A value is trying to be set on a co

return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0.0
return_ratio 0

/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['degree'] = data['team'].map(lambda x:degree(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['doctor_yesno'] = data['degree'].map(lambda x: degree_quant(x))
/Users/catherinechen/code/bpifrance_deeptech_analysis/bpideep/feateng.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy 

encoder 0.39
return_list 0.0
encoder 0.62
return_list 0.0
encoder 0.59
return_list 0.0
encoder 0.78
return_list 0.0
encoder 1.06
return_list 0.0
encoder 3.79
return_list 0.0
encoder 0.37
feat_eng_cols 7.73


In [17]:
scoring=["test_accuracy", "test_recall", "test_f1", "test_precision"]

for metric in scoring:
    print(f"{metric} : {cv_result[metric].mean() * 100 : .2f}%")

test_accuracy :  69.21%
test_recall :  60.26%
test_f1 :  66.06%
test_precision :  74.68%


1st run : test_accuracy :  69.60%
test_recall :  62.53%
test_f1 :  67.38%
test_precision :  73.59%


2nd run (en instanciant bien le model2) : test_accuracy :  69.21%
test_recall :  60.26%
test_f1 :  66.06%
test_precision :  74.68%